# Selenium 및 BeautifulSoup을 이용한 동적 및 정적 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

코드 참고 https://www.fun-coding.org/crawl_basic2.html

## 1. 남성 옷

______________________________

In [ ]:
# 패션 스타일 카테고리별 데이터프레임 생성
american_casual_m = pd.DataFrame(columns=['title','img', 'link'])
casual_m = pd.DataFrame(columns=['title','img', 'link'])
dandy_m = pd.DataFrame(columns=['title','img', 'link'])
formal_m = pd.DataFrame(columns=['title','img', 'link'])
sports_m = pd.DataFrame(columns=['title','img', 'link'])
street_m = pd.DataFrame(columns=['title','img', 'link'])

### 1-1. 크롤링 함수 정의

In [ ]:
#상세품목 크롤링 함수
def crawling_detailproducts(product_links):
    #prouct_links: 상세품목 링크 리스트
    details = []
    for i in product_links:
        res = requests.get(i,  headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(res.content, 'html.parser')

        #이미지
        image = soup.find_all('div','product-img')
        img_link = str(image).split('"')[11]
        #상품명
        prd_title = soup.find_all('span','product_title')
        prd_title = str(prd_title).split('<')[2].split('>')[1]
        #가격
        price = soup.find_all('span', 'product_article_price')
        if '원' in str(price).split('>')[2].split('<')[0]:
            price = str(price).split('>')[2].split('<')[0]
        else:
            price = str(price).split('\n')[1]
        #브랜드명
        b = soup.find_all('p', 'product_article_contents')
        brand = str(b).split('<strong>')[1].split('>')[1].split('<')[0]
        #품번
        product_num = str(b).split('<strong>')[1].split('</span> ')[1].split('<')[0]
        #성별
        sex = soup.find_all('span', 'txt_gender')
        sex_str = str(b).split('<span>')
        if len(sex_str) < 3:
            sex = sex_str[1].split('<')[0]
        else:
            sex = '남,여'
        #해시태그
        hashtag = soup.find_all('a', 'listItem')
        tag = str(hashtag).split('"')
        hash =[]
        j=4
        while j < len(tag):
            t = tag[j].split('#')[1].split('<')[0]
            hash.append(t)
            j += 4

        #합치기
        detail = [prd_title, img_link, price, brand, product_num, sex, hash]
        details.append(detail)
       
    return details
        

남성 탭을 클릭해야 남성 페이지가 도출되므로 동적크롤링 실시

In [ ]:
# 남성 코디 크롤링
# selenium을 이용한 동적 크롤링 & Beautifulsoup을 이용한 정적 크롤링

# 크롤링 함수 생성
def crawling_musinsa(pagelink, style_name, df):
    driver = webdriver.Chrome('C:/Korea university/공모전/해커톤/크롤링/chromedriver.exe')
    driver.get(pagelink)
    time.sleep(1)

    #남성 탭 클릭
    x_path = '/html/body/div[3]/div[3]/div[1]/button[2]'
    codiforman = driver.find_element(By.XPATH, x_path)
    codiforman.click()
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    content = soup.find_all('a', 'style-list-item__link')

    #코디 제목, 링크 
    title = []
    link = []
    for i in content:
        con = str(i).split('"')
        title.append(con[7])
        li = 'https://www.musinsa.com/app/styles/views/' + con[5].split("'")[1] + '?use_yn_360=&style_type=' + style_name + '&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'
        link.append(li)

    df['title'] = title
    df['link'] = link


    #코디 상세페이지
    img = []
    txt_total = []
    hashtag_total = []
    product_link_total = []
    product_detail_total = []

    for i in df['link']:
        res = requests.get(i,  headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(res.content, 'html.parser')
        #이미지
        image = soup.find_all('img','detail_img')
        imagelist = []
        for j in image:
            imagelist.append(str(j).split('"')[5])
        #내용
        txt = soup.find_all('p', 'styling_txt')
        txt_final = str(txt).split('>')[1].split('<')[0]
        #해시태그
        hashtag = soup.find_all('div', 'ui-tag-list')
        tag = str(hashtag).split('"')
        hash =[]
        j=7
        while j < len(tag):
            hash.append(tag[j])
            j += 8
        #상품 url
        products = soup.find_all('a', 'brand_item')
        product_link=[]
        for p in range(10):
            if (4*p+3) > len(str(products).split('"')):
                break
            product_li = 'https://www.musinsa.com' + str(products).split('"')[4*p+3]
            product_link.append(product_li)
        #상세품목 정보 크롤링
        product_detail = crawling_detailproducts(product_link)

        #저장
        img.append(imagelist)
        txt_total.append(txt_final)
        hashtag_total.append(hash)
        product_link_total.append(product_link)
        product_detail_total.append(product_detail)
        

    df['img'] = img
    df['txt'] = txt_total
    df['hashtag'] = hashtag_total
    df['product_links'] = product_link_total
    df['products_info'] = product_detail_total

    driver.close()


### 1-2. 각 카테고리 별 크롤링

In [ ]:
#american_casual_m에 대해 크롤링
pagelink = 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=americancasual&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'
crawling_musinsa(pagelink, 'americancasual', american_casual_m)
american_casual_m.head()

C:\Users\minje\AppData\Local\Temp\ipykernel_13920\2006456213.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Korea university/공모전/해커톤/크롤링/chromedriver.exe')


,title,img,link,txt,hashtag,product_links,products_info
0,가볍지만 멋스럽게,[//image.msscdn.net/images/style/detail/24745/...,https://www.musinsa.com/app/styles/views/24745...,깔끔한 디자인의 터틀넥 스웨터에 퀼팅 재킷을 연출하고 코튼 팬츠로 마무리한 아메리칸...,"[퀼팅, 핀턱, 와이드핏, 뉴트럴톤, 톤온톤, 아메카지, 시티보이룩, 겨울, 아메리...","[https://www.musinsa.com/app/goods/2268307/0, ...","[[코튼 실루엣 팬츠 베이지, //image.msscdn.net/images/goo..."
1,서머 빈티지,[//image.msscdn.net/images/style/detail/26681/...,https://www.musinsa.com/app/styles/views/26681...,미니멀한 피케 티셔츠에 와이드 팬츠를 코디하고 버킷햇으로 마무리한 아메리칸 캐주얼 룩,"[오버핏, 벌룬핏, 시티보이룩, 꾸안꾸, 뉴트럴톤, 편안한, 컬러블록, 봄, 아메리...","[https://www.musinsa.com/app/goods/2500338/0, ...","[[오버 핏 피케 셔츠 [라이트 베이지], //image.msscdn.net/ima..."
2,톤온톤 조합,[//image.msscdn.net/images/style/detail/24616/...,https://www.musinsa.com/app/styles/views/24616...,브라운 컬러 셔츠와 함께 코듀로이 점퍼를 스타일링하고 와이드 팬츠로 꾸민 아메리칸 ...,"[뉴트럴톤, 톤온톤, 코듀로이, 와이드핏, 겨울, 아메리칸 캐주얼, 클락스, 트릴리...","[https://www.musinsa.com/app/goods/2194630/0, ...","[[코듀로이 후디드 봄버 점퍼 (브라운), //image.msscdn.net/ima..."
3,미니멀 감성,[//image.msscdn.net/images/style/detail/26766/...,https://www.musinsa.com/app/styles/views/26766...,로고를 더한 반팔 티셔츠에 숏 팬츠를 매치하고 에코백을 들어 완성한 아메리칸 캐주얼 룩,"[레이어드룩, 로고플레이, 오버핏, 컬러포인트, 체크, 시티보이룩, 봄, 아메리칸 ...","[https://www.musinsa.com/app/goods/2024232/0, ...","[[시티보이 빅오버 옥스포드 셔츠_Royal Blue, //image.msscdn...."
4,멋스러운 모습,[//image.msscdn.net/images/style/detail/26765/...,https://www.musinsa.com/app/styles/views/26765...,아노락에 오버 핏 숏 팬츠를 매치하고 크로스 백을 포인트로 들어 마무리한 아메리칸 ...,"[뉴트럴톤, 톤온톤, 오버핏, 시티보이룩, 캠핑, 봄, 아메리칸 캐주얼, 그라미치,...","[https://www.musinsa.com/app/goods/1863727/0, ...","[[[COOLTECH] 라이트 반팔 아노락_SPJJC25C22, //image.ms..."


In [ ]:
american_casual_m.to_csv('C:/Korea university/공모전/해커톤/크롤링/american_casual_man.csv')

In [ ]:
pagelink = 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=casual&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'
crawling_musinsa(pagelink, 'casual', casual_m)
casual_m.head()

C:\Users\minje\AppData\Local\Temp\ipykernel_13920\2006456213.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Korea university/공모전/해커톤/크롤링/chromedriver.exe')


,title,img,link,txt,hashtag,product_links,products_info
0,나쁠게 없어!,[//image.msscdn.net/images/style/detail/8679/d...,https://www.musinsa.com/app/styles/views/8679?...,\n\n\n\n\n\n\n\n트러커 재킷과 스트라이프 패턴 티셔츠를 코디하고 데님 ...,"[봄, 캐주얼, 토피, 반스, 네스티킥, 데이데어, 데님 팬츠, 안경 소품, 액자/...","[https://www.musinsa.com/app/goods/952064/0, h...","[[TRUCKER JACKET(2COLOR), //image.msscdn.net/i..."
1,스포티룩,[//image.msscdn.net/images/style/detail/8412/d...,https://www.musinsa.com/app/styles/views/8412?...,\n\n\n\n\n\n\n\n화사한 아이보리 컬러의 스웨트셔츠와 트랙 팬츠를 매치하...,"[봄, 캐주얼, 아디다스, 디스이즈네버댓, 패션스니커즈화, 캡/야구 모자, 반소매 ...","[https://www.musinsa.com/app/goods/957879/0, h...","[[릴랙스 오버사이즈 크루넥 - 베이지 / CM1408, //image.msscdn..."
2,꾸러기 룩,[//image.msscdn.net/images/style/detail/14460/...,https://www.musinsa.com/app/styles/views/14460...,\n오버사이즈 핏의 블레이저와 쇼츠를 매치하고 볼캡과 타이로 포인트를 더한 캐주얼 룩,"[봄, 캐주얼, 무신사 스탠다드, 노베스타, 바잘, 넥타이, 슈트 팬츠/슬랙스, 선...","[https://www.musinsa.com/app/goods/1283757/0, ...","[[오버사이즈 블레이저 [블랙], //image.msscdn.net/images/g..."
3,심플한데 멋져!,[//image.msscdn.net/images/style/detail/22571/...,https://www.musinsa.com/app/styles/views/22571...,럭비 스타일의 스웨트셔츠에 크림 컬러 팬츠를 매치하고 캔버스 백을 들어 완성한 캐주얼 룩,"[캠퍼스, 꾸안꾸, 와이드핏, 여름, 캐주얼, 블러프, 119레오, 카고브로스, 이...","[https://www.musinsa.com/app/goods/2038497/0, ...","[[[기모ver.]헤비 기모 오버 럭비 맨투맨_Midnight Blue, //ima..."
4,가벼운 발걸음,[//image.msscdn.net/images/style/detail/13457/...,https://www.musinsa.com/app/styles/views/13457...,\n\n\n\n\n\n\n\n네이비 컬러 재킷 위에 숏패딩을 매치하고 와이드 데님 ...,"[겨울, 캐주얼, 무신사 스탠다드, 페이탈리즘, 아디다스, 숏패딩/숏헤비 아우터, ...","[https://www.musinsa.com/app/goods/912315/0, h...","[[다운 푸퍼 숏 패딩 재킷 [라이트 그레이], //image.msscdn.net/..."


In [ ]:
casual_m.to_csv('C:/Korea university/공모전/해커톤/크롤링/casual_man.csv')

In [ ]:
pagelink = 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=dandy&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'
crawling_musinsa(pagelink, 'dandy', dandy_m)

C:\Users\minje\AppData\Local\Temp\ipykernel_13920\2006456213.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Korea university/공모전/해커톤/크롤링/chromedriver.exe')


In [ ]:
dandy_m.to_csv('C:/Korea university/공모전/해커톤/크롤링/dandy_man.csv')

In [ ]:
pagelink = 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=formal&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'
crawling_musinsa(pagelink, 'formal', formal_m)

C:\Users\minje\AppData\Local\Temp\ipykernel_13920\2006456213.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Korea university/공모전/해커톤/크롤링/chromedriver.exe')


In [ ]:
formal_m.to_csv('C:/Korea university/공모전/해커톤/크롤링/formal_man.csv')

In [ ]:
pagelink = 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=sports&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'
crawling_musinsa(pagelink, 'sports', sports_m)

C:\Users\minje\AppData\Local\Temp\ipykernel_13920\2006456213.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Korea university/공모전/해커톤/크롤링/chromedriver.exe')


In [ ]:
sports_m.to_csv('C:/Korea university/공모전/해커톤/크롤링/sports_man.csv')

In [ ]:
pagelink = 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=street&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'
crawling_musinsa(pagelink, 'street', street_m)

C:\Users\minje\AppData\Local\Temp\ipykernel_13920\2006456213.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Korea university/공모전/해커톤/크롤링/chromedriver.exe')


In [ ]:
street_m.to_csv('C:/Korea university/공모전/해커톤/크롤링/street_man.csv')

## 2. 여성 옷

In [ ]:
casual_w = pd.DataFrame(columns=['title','img', 'link'])
chic_w = pd.DataFrame(columns=['title','img', 'link'])
formal_w = pd.DataFrame(columns=['title','img', 'link'])
girlish_w = pd.DataFrame(columns=['title','img', 'link'])
sports_w = pd.DataFrame(columns=['title','img', 'link'])
street_w = pd.DataFrame(columns=['title','img', 'link'])
golf_w = pd.DataFrame(columns=['title','img', 'link'])
retro_w = pd.DataFrame(columns=['title','img', 'link'])
romantic_w = pd.DataFrame(columns=['title','img', 'link'])

### 2-1. 크롤링 함수 정의

여성은 musinsa를 wusinsa로 링크를 변경하면 바로 도출되므로 동적크롤링이 필요없다.

In [ ]:
# 여성 코디 크롤링
# selenium을 이용한 동적 크롤링 & Beautifulsoup을 이용한 정적 크롤링

# 크롤링 함수 생성
def crawling_wusinsa(pagelink, style_name, df):
    res = requests.get(pagelink,  headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(res.content, 'html.parser')
    content = soup.find_all('a', 'style-list-item__link')

    #코디 제목, 링크 
    title = []
    link = []
    for i in content:
        con = str(i).split('"')
        title.append(con[7])
        li = 'https://www.wusinsa.com/app/styles/views/' + con[5].split("'")[1] + '?use_yn_360=&style_type=' + style_name + '&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'
        link.append(li)

    df['title'] = title
    df['link'] = link


    #코디 상세페이지
    img = []
    txt_total = []
    hashtag_total = []
    product_link_total = []
    product_detail_total = []

    for i in df['link']:
        res = requests.get(i,  headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(res.content, 'html.parser')
        #이미지
        image = soup.find_all('img','detail_img')
        imagelist = []
        for j in image:
            imagelist.append(str(j).split('"')[5])
        #내용
        txt = soup.find_all('p', 'styling_txt')
        txt_final = str(txt).split('>')[1].split('<')[0]
        #해시태그
        hashtag = soup.find_all('div', 'ui-tag-list')
        tag = str(hashtag).split('"')
        hash =[]
        j=7
        while j < len(tag):
            hash.append(tag[j])
            j += 8
        #상품 url
        products = soup.find_all('a', 'brand_item')
        product_link=[]
        for p in range(10):
            if (4*p+3) > len(str(products).split('"')):
                break
            product_li = 'https://www.wusinsa.com' + str(products).split('"')[4*p+3]
            product_link.append(product_li)
        #상세품목 정보 크롤링
        product_detail = crawling_detailproducts(product_link)
        
        #저장
        img.append(imagelist)
        txt_total.append(txt_final)
        hashtag_total.append(hash)
        product_link_total.append(product_link)
        product_detail_total.append(product_detail)
        

    df['img'] = img
    df['txt'] = txt_total
    df['hashtag'] = hashtag_total
    df['product_links'] = product_link_total
    df['products_info'] = product_detail_total

### 2-2. 각 카테고리 별 크롤링

In [ ]:
pagelink = 'https://www.wusinsa.com/app/styles/lists?use_yn_360=&style_type=casual&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'
crawling_wusinsa(pagelink, 'casual', casual_w)
casual_w.to_csv('C:/Korea university/공모전/해커톤/크롤링/여성/casual_woman.csv')

pagelink = 'https://www.wusinsa.com/app/styles/lists?use_yn_360=&style_type=chic&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'
crawling_wusinsa(pagelink, 'chic', chic_w)
chic_w.to_csv('C:/Korea university/공모전/해커톤/크롤링/여성/chic_woman.csv')

pagelink = 'https://www.wusinsa.com/app/styles/lists?use_yn_360=&style_type=formal&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'
crawling_wusinsa(pagelink, 'formal', formal_w)
formal_w.to_csv('C:/Korea university/공모전/해커톤/크롤링/여성/formal_woman.csv')

pagelink = 'https://www.wusinsa.com/app/styles/lists?use_yn_360=&style_type=girlish&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'
crawling_wusinsa(pagelink, 'girlish', girlish_w)
girlish_w.to_csv('C:/Korea university/공모전/해커톤/크롤링/여성/girlish_woman.csv')

pagelink = 'https://www.wusinsa.com/app/styles/lists?use_yn_360=&style_type=sports&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'
crawling_wusinsa(pagelink, 'sports', sports_w)
sports_w.to_csv('C:/Korea university/공모전/해커톤/크롤링/여성/sports_woman.csv')

pagelink = 'https://www.wusinsa.com/app/styles/lists?use_yn_360=&style_type=street&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'
crawling_wusinsa(pagelink, 'street', street_w)
street_w.to_csv('C:/Korea university/공모전/해커톤/크롤링/여성/street_woman.csv')

pagelink = 'https://www.wusinsa.com/app/styles/lists?use_yn_360=&style_type=golf&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'
crawling_wusinsa(pagelink, 'golf', golf_w)
golf_w.to_csv('C:/Korea university/공모전/해커톤/크롤링/여성/golf_woman.csv')

pagelink = 'https://www.wusinsa.com/app/styles/lists?use_yn_360=&style_type=retro&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'
crawling_wusinsa(pagelink, 'retro', retro_w)
retro_w.to_csv('C:/Korea university/공모전/해커톤/크롤링/여성/retro_woman.csv')

pagelink = 'https://www.wusinsa.com/app/styles/lists?use_yn_360=&style_type=romantic&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=1'
crawling_wusinsa(pagelink, 'romantic', romantic_w)
romantic_w.to_csv('C:/Korea university/공모전/해커톤/크롤링/여성/romantic_woman.csv')